In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as plt

#Import cranial database 
df = pd.read_csv('../../datasources/fdb/postcranial.csv')
#This shuffles the rows.
df = df.sample(frac=1)

#Drop categorical data. We'll focus only on measurement data.
del df['DB']
del df['Item']
del df['ID']
del df['ContNum']
del df['FDN']
del df['Pop']
del df['PopSex']
del df['Ethnicity'] #Probably really important.
del df['BirthYear']
del df['Age'] #Probably really important.
del df['Comments']

#This col is basically empty and disturbs standardization.
del df["MOW"]

rows, cols = df.shape 
print("Starting with", rows, "individuals and", cols, "features.")

KeyError: 'Ethnicity'

In [ ]:
#Consider our target col, Sex.
print("Unique entries.")
df['Sex'].value_counts(dropna=False)

In [ ]:
#Clean N and Nan entries
df = df[~(df.Sex.str.contains("N") == True)]
df = df.dropna(subset = ['Sex'])

#Move from object to float.
df['Sex'] = df['Sex'].map({'M':1.0, 'F':0.})
df.describe()

In [ ]:
#Split into data and labels.
X = df.drop(['Sex'], axis=1)
y = df[['Sex']]

In [ ]:
#Split train test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.isnull().sum().sum()

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#Standardize df. Can't use scikit's scale because it doesn't like Nan.
X_train_mean = X_train.mean()
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / (X_train_std)
#Intentionally using train stats for test set.
X_test = ( X_test - X_train_mean) / (X_train_std)
X_train.describe()

In [ ]:
#The imputer. I haven't looked closely at this algo. Try other variations.
from fancyimpute import KNN
X_train_filled = pd.DataFrame(KNN(k=3).complete(X_train))
X_test_filled = pd.DataFrame(KNN(k=3).complete(X_test))

In [ ]:
#Num zero entries
(X_train_filled == 0).astype(int).sum().sum(),(X_test_filled == 0).astype(int).sum().sum()

In [ ]:
#Impute zeros with mean. Could do this better.
#Think this introduces an improbable bug when the train set
#Has representation in a col that the test set doesn't or 
#Vice versa. Ignoring it for now. 
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values=0, strategy='mean', axis=0)
X_train_filled = pd.DataFrame(imp.fit_transform(X_train_filled))
X_train_filled = pd.DataFrame(imp.transform(X_train_filled))
X_test_filled = pd.DataFrame(imp.fit_transform(X_test_filled))

In [ ]:
#No zero values
(X_train_filled == 0).astype(int).sum().sum(), (X_test_filled == 0).astype(int).sum().sum()

In [ ]:
#Notice that the training (and testing) set is no longer standardized
X_train_filled.describe()

In [ ]:
#Should we standardize again?
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train_filled)
X_train_filled = pd.DataFrame(scaler.transform(X_train_filled))
#Uses test stats
X_test_filled = pd.DataFrame(scaler.transform(X_test_filled))

In [ ]:
#Save to disk
X_train_filled.to_csv("data/X_train_preproc.csv", index=False)
#Ravel strips col name and reindexes
pd.DataFrame(np.ravel(y_train)).to_csv("data/y_train_preproc.csv", index=False)

X_test_filled.to_csv("data/X_test_preproc.csv", index=False)
pd.DataFrame(np.ravel(y_test)).to_csv("data/y_test_preproc.csv", index=False)